# Sounder SIPS L1B PGE Interface

In [ ]:
import os, sys, shutil
import re
from glob import glob
import logging
import subprocess
from pprint import pformat

In [ ]:
from xml.etree import ElementTree
from xml.etree.ElementTree import Element, tostring
from xml.dom import minidom

## Execution Parameters

In [ ]:
# Location of input L0 files
input_path = "/pge/in"

# Where PGE output files and log files get written
output_path = "/pge/out"

# Location of dem and mcf static files
data_static_path = "/tmp/static"

# Enable verbose logging
verbose = True

## Constants

In [ ]:
# Where PGE static config files
config_static_path = "/pge/static"

# Source XML file to be modified with execution parameters
config_template_filename = "/pge/static/pge_config_template.xml"

# Where config file gets written
config_output_filename = os.path.join(output_path, "l1b_config.xml")

# Name of output log filename
log_filename = os.path.join(output_path, "L1BMw_main.log")

# Location of PGE executable
pge_executable = "/pge/bin/L1BMw_main"

# Location of MetExtractor executable
met_extractor_executable = "/pge/bin/MetExtractor"

## Set up Logging

In [ ]:
if verbose:
    logging.basicConfig(level=logging.DEBUG)
else:
    logging.basicConfig(level=logging.INFO)
    
logger = logging.getLogger("PGE Wrapper")

## Output Path Creation

In [ ]:
# Make sure the output directory exists
if not os.path.exists(output_path):
    logging.info(f"Creating missing output directory: {output_path}")
    os.path.makedirs(output_path)

## Identify Input Files

In [ ]:
# Find input files recursively
input_filenames = sorted(glob(os.path.join(input_path, "**", "*.nc"), recursive=True))

## Create XML Configuration

In [ ]:
# Parse configuration XML
config_root = ElementTree.parse(config_template_filename).getroot()

In [ ]:
# Modify input filenames
inp_file_elem = config_root.find("./group[@name='InputProductFiles']")

if inp_file_elem is None:
    raise Exception(f"Could not find InputProductFiles group in XML config template: {config_template_filename}")

vector_elem = inp_file_elem.find(f"./vector[@name='InputL1aFiles']")

for fn_elem, inp_filename in zip(vector_elem, input_filenames):
    fn_elem.text = inp_filename

In [ ]:
# Modify path for output filenames
out_file_elem = config_root.find("./group[@name='OutputProductFiles']/vector")

if out_file_elem is None:
    raise Exception(f"Could not find OutputProductFiles group in XML config template: {config_template_filename}")
    
output_filenames = []
for fn_elem in out_file_elem:
    fn_elem.text = os.path.join(output_path, os.path.basename(fn_elem.text))
    output_filenames.append(fn_elem.text)

In [ ]:
# Modify SFIF filename path
sfif_elem = config_root.find("./group[@name='StaticFileIdentificationFiles']/scalar")
sfif_elem.text = os.path.join(config_static_path, os.path.basename(sfif_elem.text))

In [ ]:
# Modify MonitorPath
mon_path_elem = config_root.find(".//scalar[@name='MonitorPath']")
mon_path_elem.text = output_path

In [ ]:
schema_fn = config_root.attrib['{http://www.w3.org/2001/XMLSchema-instance}noNamespaceSchemaLocation']

config_root.attrib['{http://www.w3.org/2001/XMLSchema-instance}noNamespaceSchemaLocation'] = \
    os.path.join(config_static_path, os.path.basename(schema_fn))

In [ ]:
# Write created config
logger.info(f"Writing config file: {config_output_filename}")

with open(config_output_filename, mode = 'w', encoding = 'utf-8') as output:
    rough = tostring(config_root, 'utf-8')
    reparsed = minidom.parseString(rough)
    pretty_xml = reparsed.toprettyxml(indent='  ', newl='')
    output.write(pretty_xml)

## Create L1B Template Files

In [ ]:
# Open SFIF file to locate template L1B filename
sfif_root = ElementTree.parse(sfif_elem.text).getroot()

In [ ]:
tmpl_elem = sfif_root.find("./group[@name='StaticAuxiliaryInputFiles']/scalar[@name='L1bMwTemplate']")
l1b_template_fn = tmpl_elem.text

for out_fn in output_filenames:
    logger.info(f"Creating template L1B output file: {out_fn}")
    shutil.copyfile(l1b_template_fn, out_fn)

## Run L1BMw_main PGE executable

In [ ]:
# Run L1BMw_main PGE executable

# Change to out path so that any PGS temporary files are written there
os.chdir(output_path)

l1b_cmd = pge_executable + ' ' + config_output_filename + ' ' + log_filename

logger.info(f"Running PGE executable: {l1b_cmd}")

l1b_status = subprocess.run(l1b_cmd, shell=True)

if (l1b_status.returncode != 0):
    raise Exception(f"Execution of PGE resulting in non zero exit status: {l1b_status}, check log file for details: {log_filename}")

## Run Met Extractor

In [ ]:
# Extract from the SFIF file the path to 
met_const_elem = sfif_root.find("./group[@name='OutputProductConfiguration']//scalar[@name='MetFileConstants']")
met_const_filename = met_const_elem.text

met_mapping_elem = sfif_root.find("./group[@name='OutputProductConfiguration']//scalar[@name='MetFileMappings']")
met_mapping_filename = met_mapping_elem.text

In [ ]:
# Write pev file for to capture config parameters to Product metadata

pev_filename = os.path.join(output_path, 'spdc.pev')
omit_list = ['ProductionDateTime', 'ProductionLocation', 'ProductionLocationCode', 'CollectionLabel', 'NodeInfo']
group_path_list = ['JobIdentification', 'SCFIdentification']

In [ ]:
def extract_config_group_to_pev(config_root, group_path, pev_file, omit_list):
    
    scalar_fields = config_root.findall(f".//group[@name='{group_path}']/scalar")
    
    for field in scalar_fields:
        name = field.attrib['name']
        value = field.text
        
        if name not in omit_list:
            pev_file.write(f"{name}={value}\n")

In [ ]:
with open(pev_filename,'w') as pev_file:
    for group_path in group_path_list:
        extract_config_group_to_pev(config_root, group_path, pev_file, omit_list)
    pev_file.close()

In [ ]:
for out_file in output_filenames:

    # make sure to write abspath into met file
    met_cmd = met_extractor_executable + ' -Ddata.file.reader.hdf5.data.types.map.file=' + met_mapping_filename + \
        ' --dataFile -file ' + os.path.abspath(out_file) + ' -reader SipsNcHDF5FileReader ' + \
        ' --metFile -toFile ' + out_file + '.cas -writer XmlCasWriter ' + \
        ' --supportFile -file ' + met_const_filename + ' -reader PropEqValFileReader ' + \
        ' --supportFile -file ' + pev_filename + ' -reader PropEqValFileReader -Ddebug=true'

    met_status = subprocess.run(met_cmd, shell=True)
    if (met_status.returncode != 0):
        raise Exception(f"Error executing MetExtractor command: {met_cmd}")